# Baseline vs m6A - setting A - 

# Imports

In [1]:
# imports
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%load_ext line_profiler
import os
import sys
import importlib

import numpy as np
from sklearn.utils import compute_class_weight
import pandas as pd
from tensorflow import keras 

sys.path.append('/home/ubuntu/')
os.chdir('/home/ubuntu/')
import src.model
importlib.reload(src.model)
import src.model as mdl

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


2023-06-06 10:49:35.249227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 10:49:36.025750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#all paths
datapath = '/mnt/data/'
resultspath = '/mnt/data/results/'
predictionspath = '/mnt/data/results/all_predictions/'
plotspath = '/mnt/data/results/plots/'


%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%load_ext line_profiler

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

def prepare_raw_dataset_m6A(path, current_folder):
    augmentation= False
    print('Starting preparation of the dataset for %s' % current_folder)

    folder = current_folder #RBP
    directory2 = os.listdir(f'{path}{folder}')

    seq_list_pos = np.zeros((1, 400, 5))
    seq_list_neg1 = np.zeros((1, 400, 5))
    seq_list_neg2 = np.zeros((1, 400, 5))
    seq_list_pos_test = np.zeros((1, 400, 5))
    seq_list_neg1_test = np.zeros((1, 400, 5))
    seq_list_neg2_test = np.zeros((1, 400, 5))

    for folder2 in directory2:  # fold0, fold1..
        #directory3 = os.listdir(f'{path}{folder}/{folder2}')

        if folder2 != "fold-4":  # to be split into training and validation
            seq_list_pos_file, seq_list_neg1_file, seq_list_neg2_file = mdl.getonehotencoded_files(
                f'{path}{folder}/{folder2}', augmentation)

            seq_list_pos = np.vstack((seq_list_pos, seq_list_pos_file))
            seq_list_neg1 = np.vstack((seq_list_neg1, seq_list_neg1_file))
            seq_list_neg2 = np.vstack((seq_list_neg2, seq_list_neg2_file))

        if folder2 == "fold-4":  # for testing
            seq_list_pos_test_file, seq_list_neg1_test_file, seq_list_neg2_test_file = mdl.getonehotencoded_files(
                f'{path}{folder}/{folder2}', augmentation)

            seq_list_pos_test = np.vstack((seq_list_pos_test,
                                           seq_list_pos_test_file))
            seq_list_neg1_test = np.vstack((seq_list_neg1_test,
                                            seq_list_neg1_test_file))
            seq_list_neg2_test = np.vstack((seq_list_neg2_test,
                                            seq_list_neg2_test_file))

    seq_list_pos = np.delete(seq_list_pos, 0, 0)
    seq_list_neg1 = np.delete(seq_list_neg1, 0, 0)
    seq_list_neg2 = np.delete(seq_list_neg2, 0, 0)

    seq_list_pos_test = np.delete(seq_list_pos_test, 0, 0)
    seq_list_neg1_test = np.delete(seq_list_neg1_test, 0, 0)
    seq_list_neg2_test = np.delete(seq_list_neg2_test, 0, 0)
    return 


%lprun -f prepare_raw_dataset_m6A(path,'RBM15_HepG2')

# Comparison 

In [ ]:
# Preparation of the dataset for setting A with bound and unbound sequences ( pos and neg1) 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)

for folder in directory1:
    #if folder in list_high_meth :#to be done for high methylation RBPs
    if folder == 'PCBP1_HepG2':
        #try:
        # Encoding input files --> no augmentation
        path_figure = f'{plotspath}summary_metrics/{folder}/'
        pos, neg1, neg2, pos_t, neg1_t, neg2_t = mdl.prepare_raw_dataset_m6A(path, folder, False)

        # JUST RBP INFORMATION
        output_folder = '_baseline_A'

        # preparing the files and labels following the setting A rules --> 4 channels, no downsampling
        pos_filt, neg1_filt, pos_t_filt, neg1_t_filt, labels_pos, labels_neg1, labels_pos_t, labels_neg1_t = mdl.settingA(
            pos, neg1, pos_t, neg1_t, 4, True)

        # finalizing the dataset merging positive and the chosen negative ( either unbound - neg1- or bound to other RBPs -neg2- )
        x_train, y_train, x_val, y_val, x_test, y_test = mdl.finalize_dataset(
            folder, pos_filt, neg1_filt, pos_t_filt, neg1_t_filt,
            labels_pos, labels_neg1, labels_pos_t, labels_neg1_t, path_figure, output_folder)

        # creating the baseline model with the right input shape
        #model = keras.models.load_model(f'{resultspath}best_models/best_hps_model{folder}')
        model = mdl.create_baseline_standard()
        # training the model on the new datasets
        y_labels = np.reshape(y_train, (len(y_train), ))
        class_weights = compute_class_weight(class_weight="balanced",
                                             classes=np.unique(y_labels),
                                             y=y_labels)
        class_weights = dict(zip(np.unique(y_train), class_weights))

        mdl.training_model(model, class_weights, x_train, y_train, x_val, y_val,
                           x_test, y_test, folder, resultspath, output_folder, path_figure)  # trains the model with best hyperparameters 

        # evaluation and storage of the predicted labels
        model_path = (f'{resultspath}best_models/{output_folder}/%s' %folder)
        mdl.testing_model(model_path, folder, x_test, y_test,
                          predictionspath, output_folder, path_figure)

        # M6A DATA INCLUDED
        output_folder = '_setting_A'

        # preparing the files and labels following the setting A rules --> 5 channels, no downsampling
        pos_filt, neg1_filt, pos_t_filt, neg1_t_filt, labels_pos, labels_neg1, labels_pos_t, labels_neg1_t = mdl.settingA(
            pos, neg1, pos_t, neg1_t, 5, True)

        # finalizing the dataset merging positive and the chosen negative (either unbound - neg1- or bound to other RBPs -neg2-)
        x_train, y_train, x_val, y_val, x_test, y_test = mdl.finalize_dataset(
            folder, pos_filt, neg1_filt, pos_t_filt, neg1_t_filt,
            labels_pos, labels_neg1, labels_pos_t, labels_neg1_t, path_figure, output_folder)

        # creating the baseline model with the right input shape
        #model = keras.models.load_model(f'{resultspath}best_models/best_hps_model_5{folder}')
        model = mdl.create_baseline_model((400, 5))
        # training the model on the new datasets
        y_labels = np.reshape(y_train, (len(y_train), ))
        class_weights = compute_class_weight(class_weight="balanced",
                                             classes=np.unique(y_labels),
                                             y=y_labels)
        class_weights = dict(zip(np.unique(y_train), class_weights))

        mdl.training_model(model, class_weights, x_train, y_train, x_val, y_val,
                           x_test, y_test, folder, resultspath, output_folder, path_figure)  # trains the model with best hyperparameters 

        # evaluation and storage of the predicted labels
        model_path = (f'{resultspath}best_models/{output_folder}/%s' % folder)
        mdl.testing_model(model_path, folder, x_test, y_test,
                          predictionspath, output_folder, path_figure)
        #except:  # I don't remember which type of error was raised 
            #print(f'Exception occurred in {folder}')
            #continue

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
Starting preparation of the dataset for RBM15_HepG2
